In [19]:
import ast
import numpy as np
import pandas as pd

import os
import shutil
import tarfile

from scipy import ndimage
import mcubes
import trimesh
from scipy.ndimage import zoom
from plyfile import PlyData, PlyElement 

from auxiliary import values as v
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.data import imaging

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [51]:
def median_3d_array(img, disk_size=3):
    """
    Apply a median filter to a 3D image. (Stack by stack)
    :param img: Input 3D image
    :param disk_size: Size of the disk structuring element
    :return: Image with median filter applied
    """
    from skimage import morphology
    
    
    if len(img.shape) == 4:
        img = img[:, :, :, 0]
    # return ndimage.median_filter(img, size=disk_size)
    return morphology.binary_closing(img, morphology.ball(disk_size))
    

def post_process(mesh):
    """
    Post-process a mesh to ensure it is watertight.
    """
    # if not isinstance(mesh, trimesh.Trimesh):
    #     mesh = trimesh.Trimesh(vertices=mesh['vertices'], faces=mesh['faces'])
    
    mesh.remove_degenerate_faces()
    mesh.remove_duplicate_faces()
    mesh.remove_infinite_values()
    mesh.remove_unreferenced_vertices()
    
    max_iters = 10
    is_watertight = mesh.is_watertight
    while not is_watertight and max_iters > 0:
        mesh.fill_holes()
        
        is_watertight = mesh.is_watertight
        max_iters -= 1
    return mesh


def marching_cubes(img, metadata, n_faces=10000):
    """
    Generate a mesh from a binary image using the marching cubes algorithm.
    :param img: Input binary 3D image
    :param metadata: Metadata containing the resolution of the image
    :param n_faces: Number of faces of the mesh
    :return: Dictionary containing the vertices, faces and normals of the mesh
    """
    add = 10
    img = median_3d_array(img, disk_size=5)
    aux = np.zeros(np.array(img.shape) + add, dtype=np.uint8)
    aux[
        add // 2: -add // 2,
        add // 2: -add // 2,
        add // 2: -add // 2
    ] = img
    
    vert, trian = mcubes.marching_cubes(mcubes.smooth(aux), 0)
    assert len(vert) > 0 and len(trian) > 0, 'No mesh was generated'
    
    
    vert -= vert.mean(axis=0)
    vert *= np.array([metadata['x_res'], metadata['y_res'], metadata['z_res']])
    
    mesh = trimesh.Trimesh(vert, trian, process=False)
    
    mesh = mesh.simplify_quadratic_decimation(n_faces)
    trimesh.smoothing.filter_laplacian(
        mesh, lamb=0.6, iterations=15,
        volume_constraint=False
    )
    
    mesh = post_process(mesh)
    
    normals = mesh.vertex_normals
    return {
        'vertices': mesh.vertices,
        'faces': mesh.faces,
        'normals': normals
    }
    
    
def run(img_path, path_out, metadata, n_faces=10000):
    img = imaging.read_image(img_path, axes='XYZ', verbose=1)
    
    # Convert binary 
    img = img > 0
    
    mesh_data = marching_cubes(img, metadata, n_faces)
    assert mesh_data is not None, 'No mesh was generated'
    
    vertices = mesh_data['vertices']
    faces = mesh_data['faces']
    normals = mesh_data['normals']
    
    vertex_dtype = [
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('nx', 'f4'), ('ny', 'f4'), ('nz', 'f4')
    ]
    vertex_data = np.empty(len(vertices), dtype=vertex_dtype)
    vertex_data['x'] = vertices[:, 0]
    vertex_data['y'] = vertices[:, 1]
    vertex_data['z'] = vertices[:, 2]
    vertex_data['nx'] = normals[:, 0]
    vertex_data['ny'] = normals[:, 1]
    vertex_data['nz'] = normals[:, 2]
    
    face_dtype = [('vertex_indices', 'i4', (3,))]
    face_data = np.empty(len(faces), dtype=face_dtype)
    face_data['vertex_indices'] = faces
    
    # Create PlyElement objects
    vertex_element = PlyElement.describe(vertex_data, 'vertex')
    face_element = PlyElement.describe(face_data, 'face')
    
    # Write the PLY file using plyfile
    mesh = PlyData([vertex_element, face_element], text=True)
    
    mesh.write(path_out)
    print('Mesh saved to:', path_out)

In [52]:
img_path = v.data_path + 'auxiliary/Miquel_Test/E47_SHF_segmentation.tif'
path_out = img_path.replace('_segmentation.tif', '_mesh.ply')

metadata = {
    'x_res' : 0.56, 
    'y_res' : 0.56, 
    'z_res' : 3
}

run(img_path, path_out, metadata)

Reading TIFF: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/auxiliary/Miquel_Test/E47_SHF_segmentation.tif
Mesh saved to: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/auxiliary/Miquel_Test/E47_SHF_mesh.ply
